# Analyze Industrial composition of municipalities in the region


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import folium
import geopandas as gpd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import math
import folium
import datetime
from os import listdir
from os.path import isfile, join

from matplotlib import gridspec
from matplotlib.backends.backend_pdf import PdfPages

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

## Read ateco 1 and ateco 2 codes and merge with their respective descriptions

In [2]:
ateco2007 = pd.read_csv('quake_data/Ateco2007.csv', sep=';')
ateco1 = ateco2007[ateco2007['Codice Ateco 2007'].str.len()==1].reset_index().drop(['index','Codice Ateco 2007'],axis=1)
ateco1.columns = ['ateco1','desc1']
ateco2 = ateco2007[ateco2007['Codice Ateco 2007'].str.len()==2].reset_index().drop('index',axis=1)
ateco2.columns = ['ateco1','ateco2','desc2']
ateco = pd.merge(ateco1,ateco2)
ateco

,ateco1,desc1,ateco2,desc2
0,A,"AGRICOLTURA, SILVICOLTURA E PESCA",01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
1,A,"AGRICOLTURA, SILVICOLTURA E PESCA",02,SILVICOLTURA ED UTILIZZO DI AREE FORESTALI
2,A,"AGRICOLTURA, SILVICOLTURA E PESCA",03,PESCA E ACQUACOLTURA
3,B,ESTRAZIONE DI MINERALI DA CAVE E MINIERE,05,ESTRAZIONE DI CARBONE (ESCLUSA TORBA)
4,B,ESTRAZIONE DI MINERALI DA CAVE E MINIERE,06,ESTRAZIONE DI PETROLIO GREGGIO E DI GAS NATURALE
...,...,...,...,...
83,S,ALTRE ATTIVITÀ DI SERVIZI,95,RIPARAZIONE DI COMPUTER E DI BENI PER USO PERS...
84,S,ALTRE ATTIVITÀ DI SERVIZI,96,ALTRE ATTIVITÀ DI SERVIZI PER LA PERSONA
85,T,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
86,T,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...


In [3]:
# aida_crateri3.csv contains companies that were founded before the earthquake (Aug. 2016) and were active when the earthquake hit

aida = pd.read_csv('quake_data/aida_crateri3.csv')
aida = aida[['Comune','Ricavi delle vendite migl EUR Ultimo anno disp.','Dipendenti Ultimo anno disp.','Stato giuridico','ateco2']]
aida.columns = ['Comune','ricavi','dipendenti','stato','ateco2']

aida['ateco2'] = aida['ateco2'].astype('string')
m = aida['ateco2'].str.len().max()
aida['ateco2'] = aida['ateco2'].str.rjust(m, '0').astype('str').str[0:2]

aida[aida['ateco2']=='02']

,Comune,ricavi,dipendenti,stato,ateco2
115,Fabriano,2010.0,3,Attiva,02
183,Fabriano,601.0,0,Attiva,02
190,Fabriano,537.0,42,Fallita,02
249,Fabriano,250.0,30,Cessata (Fusione),02
434,Fabriano,1.0,2,Attiva,02
...,...,...,...,...,...
52484,Fara in Sabina,160.0,3,Attiva,02
52750,Caramanico Terme,136.0,1,Attiva,02
52911,Magliano de' Marsi,132.0,5,Attiva,02
53167,Massa d'Albe,0.0,0,Attiva,02


In [4]:
aida_ateco = pd.merge(aida.astype('str') , ateco.astype('str') , how='left', on='ateco2')
aida_ateco.loc[aida_ateco['ricavi'] == 'n.d.','ricavi'] = np.nan
aida_ateco.loc[aida_ateco['dipendenti'] == 'n.d.','dipendenti'] = np.nan
aida_ateco['ricavi'] = aida_ateco['ricavi'].astype('float')
aida_ateco['dipendenti'] = aida_ateco['dipendenti'].astype('float')
aida_ateco[aida_ateco['ateco1']=='A']
aida_ateco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53289 entries, 0 to 53288
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Comune      53289 non-null  object 
 1   ricavi      53289 non-null  float64
 2   dipendenti  53289 non-null  float64
 3   stato       53289 non-null  object 
 4   ateco2      53289 non-null  object 
 5   ateco1      53289 non-null  object 
 6   desc1       53289 non-null  object 
 7   desc2       53289 non-null  object 
dtypes: float64(2), object(6)
memory usage: 3.7+ MB


In [5]:
aida_ateco.loc[aida_ateco['Comune']=='Acciano',:]

,Comune,ricavi,dipendenti,stato,ateco2,ateco1,desc1,desc2
52738,Acciano,120.0,2.0,Attiva,46,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...
52739,Acciano,38.0,1.0,Attiva,81,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",ATTIVITÀ DI SERVIZI PER EDIFICI E PAESAGGIO
52740,Acciano,8.0,0.0,Attiva,46,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...


**aida_ateco** dataframe contains all the companies in the region that were active when the earthquake hit, together with their ateco codes

In [6]:
# Ateco 1  sectors and some of their statistics
vc = aida_ateco['desc1'].value_counts().to_frame()
vc['p'] = vc['desc1']/vc['desc1'].sum()*100
vc

,desc1,p
COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZIONE DI AUTOVEICOLI E MOTOCICLI,9852,18.487868
ATTIVITÀ MANIFATTURIERE,9660,18.127569
COSTRUZIONI,9443,17.720355
ATTIVITA' IMMOBILIARI,6251,11.730376
"ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE",3505,6.577342
ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZIONE,3074,5.768545
"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPORTO ALLE IMPRESE",2197,4.122802
SERVIZI DI INFORMAZIONE E COMUNICAZIONE,2085,3.912627
TRASPORTO E MAGAZZINAGGIO,1594,2.991236
"AGRICOLTURA, SILVICOLTURA E PESCA",1403,2.632814


In [7]:
vc = aida_ateco.groupby(['desc1']).aggregate({'Comune':np.size,'ricavi':np.sum,'dipendenti':np.sum}).sort_values(by='Comune',ascending=False).reset_index()
vc.columns = ['ATECO','N','REVENUE','EMPLOYEES']
vc['N %'] = (vc['N'] / sum(vc['N']) * 100).astype(int)
vc['EMPL / N'] = vc['EMPLOYEES'] / vc['N']
vc['REVENUE'] = (vc['REVENUE'] / 1000).astype(int)
vc['EMPLOYEES'] = (vc['EMPLOYEES'] / 1000).astype(int)
vc['REVENUE %'] = (vc['REVENUE'] / sum(vc['REVENUE']) * 100).astype(int)
vc['EMPLOYEES %'] = (vc['EMPLOYEES'] / sum(vc['EMPLOYEES']) * 100).astype(int)
ateco_names = ['AGRICOLTURE','REAL ESTATE','HOSPITALITY','MANUFACTURING','TECH SERVICES','WHOLE/RETAIL SALES','CONSTRUCTIONS','BUSINESS SUPPORT SERVICES','COMMUNICATION AND INFORMATION SERV.','LOGISTICS']
vc

,ATECO,N,REVENUE,EMPLOYEES,N %,EMPL / N,REVENUE %,EMPLOYEES %
0,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,9852,28354,66,18,6.724726,31,15
1,ATTIVITÀ MANIFATTURIERE,9660,36901,173,18,17.987164,40,39
2,COSTRUZIONI,9443,5379,35,17,3.784073,5,8
3,ATTIVITA' IMMOBILIARI,6251,1054,3,11,0.597504,1,0
4,"ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE",3505,2318,14,6,4.084736,2,3
5,ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZ...,3074,1101,21,5,7.016265,1,4
6,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",2197,1611,26,4,12.280382,1,6
7,SERVIZI DI INFORMAZIONE E COMUNICAZIONE,2085,1027,10,3,5.162590,1,2
8,TRASPORTO E MAGAZZINAGGIO,1594,3102,22,2,13.976788,3,5
9,"AGRICOLTURA, SILVICOLTURA E PESCA",1403,1727,9,2,6.674269,1,2


In [8]:
print(vc['ATECO'][18])

AMMINISTRAZIONE PUBBLICA E DIFESA; ASSICURAZIONE SOCIALE OBBLIGATORIA


In [9]:


ateco_names = {'COMMERCIO ALL\'INGROSSO E AL DETTAGLIO; RIPARAZIONE DI AUTOVEICOLI E MOTOCICLI':'WHOLE/RETAIL SALES',
               'ATTIVITÀ MANIFATTURIERE':'MANUFACTURING',
               'COSTRUZIONI':'CONSTRUCTIONS',
               'ATTIVITA\' IMMOBILIARI':'REAL ESTATE',
               'ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE':'TECH SERVICES',
               'ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZIONE':'HOSPITALITY',
               'NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPORTO ALLE IMPRESE':'BUSINESS SUPPORT SERVICES',
               'SERVIZI DI INFORMAZIONE E COMUNICAZIONE':'COMMUNICATION AND INFORMATION SERV.',
               'TRASPORTO E MAGAZZINAGGIO':'LOGISTICS',
               'AGRICOLTURA, SILVICOLTURA E PESCA':'AGRICOLTURE',
               'SANITA\' E ASSISTENZA SOCIALE':'HEALTHCARE',
               'ATTIVITÀ FINANZIARIE E ASSICURATIVE':'FINANCE',
               'FORNITURA DI ENERGIA ELETTRICA, GAS, VAPORE E ARIA CONDIZIONATA':'UTILITIES',
               'ISTRUZIONE':'EDUCATION',
               'FORNITURA DI ACQUA; RETI FOGNARIE, ATTIVITÀ DI GESTIONE DEI RIFIUTI E RISANAMENTO':'WASTE MGMT',
               'ALTRE ATTIVITÀ DI SERVIZI':'OTHER SERV',
               'ESTRAZIONE DI MINERALI DA CAVE E MINIERE':'MINING',
               'AMMINISTRAZIONE PUBBLICA E DIFESA; ASSICURAZIONE SOCIALE OBBLIGATORIA':'PUBLIC ADMIN',
               'ORGANIZZAZIONI ED ORGANISMI EXTRATERRITORIALI':'EXTRA',
               'ATTIVITÀ ARTISTICHE, SPORTIVE, DI INTRATTENIMENTO E DIVERTIMENTO':'ENTERTAINMENT'
              }

description = {'COMMERCIO ALL\'INGROSSO E AL DETTAGLIO; RIPARAZIONE DI AUTOVEICOLI E MOTOCICLI':'Whole and retail sales companies',
               'ATTIVITÀ MANIFATTURIERE':'Manufacturing activities',
               'COSTRUZIONI':'Constructions',
               'ATTIVITA\' IMMOBILIARI':'Real estate companies',
               'ATTIVITÀ PROFESSIONALI, SCIENTIFICHE E TECNICHE':'Professional, scientific, technical activities',
               'ATTIVITÀ DEI SERVIZI DI ALLOGGIO E DI RISTORAZIONE':'Hotels and restaurants',
               'NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPORTO ALLE IMPRESE':'Business support services, renting',
               'SERVIZI DI INFORMAZIONE E COMUNICAZIONE':'Communication and information services',
               'TRASPORTO E MAGAZZINAGGIO':'Transport and warehousing',
               'AGRICOLTURA, SILVICOLTURA E PESCA':'Agriculture, farming and fishing',
               'SANITA\' E ASSISTENZA SOCIALE':'Healthcare activities',
               'ATTIVITÀ FINANZIARIE E ASSICURATIVE':'Finance and Insurance activities',
               'FORNITURA DI ENERGIA ELETTRICA, GAS, VAPORE E ARIA CONDIZIONATA':'Utilities',
               'ISTRUZIONE':'Education',
               'FORNITURA DI ACQUA; RETI FOGNARIE, ATTIVITÀ DI GESTIONE DEI RIFIUTI E RISANAMENTO':'Waste management services',
               'ALTRE ATTIVITÀ DI SERVIZI':'Other service activities',
               'ESTRAZIONE DI MINERALI DA CAVE E MINIERE':'Mining companies',
               'AMMINISTRAZIONE PUBBLICA E DIFESA; ASSICURAZIONE SOCIALE OBBLIGATORIA':'Public administration',
               'ORGANIZZAZIONI ED ORGANISMI EXTRATERRITORIALI':'Foreign organizations',
               'ATTIVITÀ ARTISTICHE, SPORTIVE, DI INTRATTENIMENTO E DIVERTIMENTO':'Arts and entertainment'
              }


for i,row in vc.iterrows():
    if row['ATECO'] in ateco_names:
        s = f"{ateco_names[row['ATECO']]} & {row['N']}({row['N %']}\\%) & {row['REVENUE']}({row['REVENUE %']}\\%) & {row['EMPLOYEES']}({row['EMPLOYEES %']}\\%) & {int(row['EMPL / N'])} & {description[row['ATECO']]}\\\\"
        print(s)

WHOLE/RETAIL SALES & 9852(18\%) & 28354(31\%) & 66(15\%) & 6 & Whole and retail sales companies\\
MANUFACTURING & 9660(18\%) & 36901(40\%) & 173(39\%) & 17 & Manufacturing activities\\
CONSTRUCTIONS & 9443(17\%) & 5379(5\%) & 35(8\%) & 3 & Constructions\\
REAL ESTATE & 6251(11\%) & 1054(1\%) & 3(0\%) & 0 & Real estate companies\\
TECH SERVICES & 3505(6\%) & 2318(2\%) & 14(3\%) & 4 & Professional, scientific, technical activities\\
HOSPITALITY & 3074(5\%) & 1101(1\%) & 21(4\%) & 7 & Hotels and restaurants\\
BUSINESS SUPPORT SERVICES & 2197(4\%) & 1611(1\%) & 26(6\%) & 12 & Business support services, renting\\
COMMUNICATION AND INFORMATION SERV. & 2085(3\%) & 1027(1\%) & 10(2\%) & 5 & Communication and information services\\
LOGISTICS & 1594(2\%) & 3102(3\%) & 22(5\%) & 13 & Transport and warehousing\\
AGRICOLTURE & 1403(2\%) & 1727(1\%) & 9(2\%) & 6 & Agriculture, farming and fishing\\
HEALTHCARE & 1077(2\%) & 1070(1\%) & 23(5\%) & 21 & Healthcare activities\\
FINANCE & 718(1\%) & 3222(

In [10]:
# We condier only the firts 10 sectors for the main analysis (smaller sectors are less significant)
# sectors = aida_ateco['desc1'].value_counts()[0:11].index
# aida_ateco = aida_ateco.loc[aida_ateco['desc1'].isin(sectors),:]

In [11]:

def ateco_groupby(df):
    aggregation = {'Comune':[np.size],'ricavi':[np.sum],'dipendenti':[np.sum]}
    ateco0 = df.groupby(['Comune']).aggregate(aggregation).reset_index()
    ateco0.columns = ['Comune','n_imprese','tot_ricavi','tot_dipendenti']
    ateco1 = df.groupby(['Comune', 'ateco1','desc1']).aggregate(aggregation).reset_index()
    ateco1.columns = ['Comune','ateco1','desc1','n_imprese','tot_ricavi','tot_dipendenti']
    ateco2 = df.groupby(['Comune','ateco2','desc2']).aggregate(aggregation).reset_index()
    ateco2.columns = ['Comune','ateco2','desc2','n_imprese','tot_ricavi','tot_dipendenti']
    return (ateco0,ateco1,ateco2)

imprese_ateco0,imprese_ateco1,imprese_ateco2 = ateco_groupby(aida_ateco)

imprese_ateco0.loc[imprese_ateco0['Comune']=='Acciano',:]

,Comune,n_imprese,tot_ricavi,tot_dipendenti
1,Acciano,3,166.0,3.0


In [12]:
imprese_ateco1.loc[imprese_ateco1['Comune']=='Acciano',:]

,Comune,ateco1,desc1,n_imprese,tot_ricavi,tot_dipendenti
2,Acciano,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,2,128.0,2.0
3,Acciano,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",1,38.0,1.0


In [13]:
imprese_ateco2.loc[imprese_ateco2['Comune']=='Acciano',:]

,Comune,ateco2,desc2,n_imprese,tot_ricavi,tot_dipendenti
2,Acciano,46,COMMERCIO ALL'INGROSSO (ESCLUSO QUELLO DI AUTO...,2,128.0,2.0
3,Acciano,81,ATTIVITÀ DI SERVIZI PER EDIFICI E PAESAGGIO,1,38.0,1.0


**imprese_ateco1** and **imprese_ateco2** are the group_by of the companies by ateco1 and ateco2 respectively. 

In [14]:
# For each municipality we compute it industrial composition in terms of ateco codes. 
# The result will be something like: municipality X' companies are about: 50% agricolture, 40% industrial, etc.


denom = imprese_ateco0[['Comune','n_imprese','tot_ricavi','tot_dipendenti']]
denom.columns = ['Comune','n_imprese0','tot_ricavi0','tot_dipendenti0']

imprese_ateco2 = imprese_ateco2.merge(denom)
imprese_ateco2['p_imprese'] = imprese_ateco2['n_imprese'] / imprese_ateco2['n_imprese0']
imprese_ateco2['p_ricavi'] = imprese_ateco2['tot_ricavi'] / imprese_ateco2['tot_ricavi0']
imprese_ateco2['p_dipendenti'] = imprese_ateco2['tot_dipendenti'] / imprese_ateco2['tot_dipendenti0']

imprese_ateco1 = imprese_ateco1.merge(denom)
imprese_ateco1['p_imprese'] = imprese_ateco1['n_imprese'] / imprese_ateco1['n_imprese0']
imprese_ateco1['p_ricavi'] = imprese_ateco1['tot_ricavi'] / imprese_ateco1['tot_ricavi0']
imprese_ateco1['p_dipendenti'] = imprese_ateco1['tot_dipendenti'] / imprese_ateco1['tot_dipendenti0']
imprese_ateco1.loc[imprese_ateco1['Comune']=='Acciano',:]

,Comune,ateco1,desc1,n_imprese,tot_ricavi,tot_dipendenti,n_imprese0,tot_ricavi0,tot_dipendenti0,p_imprese,p_ricavi,p_dipendenti
2,Acciano,G,COMMERCIO ALL'INGROSSO E AL DETTAGLIO; RIPARAZ...,2,128.0,2.0,3,166.0,3.0,0.666667,0.771084,0.666667
3,Acciano,N,"NOLEGGIO, AGENZIE DI VIAGGIO, SERVIZI DI SUPPO...",1,38.0,1.0,3,166.0,3.0,0.333333,0.228916,0.333333


In [15]:
value = 'p_ricavi'
imprese_ateco_wide = imprese_ateco1.pivot(index='Comune',columns='desc1')[value].fillna(0)
for c in imprese_ateco_wide.columns:
    imprese_ateco_wide.rename(columns={c: ateco_names[c]},inplace=True)
imprese_ateco_wide.columns
imprese_ateco_wide = imprese_ateco_wide.reset_index()
imprese_ateco_wide.head(2)

desc1,Comune,AGRICOLTURE,OTHER SERV,PUBLIC ADMIN,REAL ESTATE,ENTERTAINMENT,HOSPITALITY,FINANCE,MANUFACTURING,TECH SERVICES,...,CONSTRUCTIONS,MINING,WASTE MGMT,UTILITIES,EDUCATION,BUSINESS SUPPORT SERVICES,EXTRA,HEALTHCARE,COMMUNICATION AND INFORMATION SERV.,LOGISTICS
0,Abbateggio,0.0,0.0,0.0,0.0,0.0,0.24878,0.0,0.0,0.0,...,0.75122,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,Acciano,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.228916,0.0,0.0,0.0,0.0


In [16]:
imprese_ateco_wide.to_csv('quake_data/imprese_ateco_wide.csv',index=False)

The result is a file with all the municipalities and their industrial composition (percentages are in terms of the fraction of the total revenue of the sector VS. total overall revenues)